In [1]:
import random
from collections import Counter

import numpy as np

from sklearn.grid_search import GridSearchCV

from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import warnings
warnings.filterwarnings('ignore')

# Load Pickled Data

In [2]:
directory = 'generated_data'
train_data = np.load('{0}/train_gray_2d.dat'.format(directory))
test_data = np.load('{0}/test_gray_2d.dat'.format(directory))

train_labels = np.load('{0}/train_labels.dat'.format(directory))
test_labels = np.load('{0}/test_labels.dat'.format(directory))

In [3]:
print('Training data shape: {0} | Training labels shape: {1}'.format(train_data.shape, train_labels.shape))
print('Testing data shape:  {0}  | Testing labels shape:  {1}'.format(test_data.shape, test_labels.shape))

Training data shape: (1000, 640) | Training labels shape: (1000,)
Testing data shape:  (100, 640)  | Testing labels shape:  (100,)


# SVM Classifier

In [ ]:
param_grid = {
            'kernel': ['rbf'],
            'C': [1, 1e1, 5e1, 1e2, 5e2],
            'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
            }

svc_grid = GridSearchCV(SVC(class_weight='balanced'), param_grid)
svc_mdl = svc_grid.fit(train_data, train_labels)

In [ ]:
print('Training score is:  {0}%'.format(svc_mdl.score(train_data, train_labels)*100))
print('Test score is: {0}%'.format(svc_mdl.score(test_data, test_labels)*100))

In [ ]:
best_parameters = svc_grid.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print('\t{0}: {1}'.format(param_name, best_parameters[param_name]))
    
print('Training score is', svc_mdl.score(train_data, train_labels))
print('Test score is', svc_mdl.score(test_data, test_labels))
print('-'*80)
print('Classification report of training data:\n', classification_report(train_labels, svc_mdl.predict(train_data)))
print('-'*80)
print('Classification report of test data:\n', classification_report(test_labels, svc_mdl.predict(test_data)))

# Logistic Regression Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
param_grid = {
            'penalty': ['l2'],
            'class_weight': ['balanced'],
            'solver': ['newton-cg', 'lbfgs', 'sag'],
            'multi_class': ['ovr', 'multinomial']
            }


#logr_grid = SGDClassifier(loss='hinge', penalty='l2', alpha=0.001, shuffle=True, verbose=0, 
#                    n_jobs=4, random_state=None, learning_rate='optimal')

logr_grid = GridSearchCV(LogisticRegression(n_jobs=-1), param_grid)
logr_mdl = logr_grid.fit(train_data, train_labels)

In [ ]:
print('Training score is', logr_mdl.score(train_data, train_labels))
print('Validation score is', logr_mdl.score(test_data, test_labels))
print('-'*80)
print('Classification report of training data:\n', classification_report(train_labels, logr_mdl.predict(train_data)))
print('-'*80)
print('Classification report of validation data:\n', classification_report(test_labels, logr_mdl.predict(test_data)))

# Decision Tree Classifier

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
pipeline = Pipeline([('clf',DecisionTreeClassifier(criterion='entropy',random_state=20000))])

parameters = {
            'criterion': ['gini', 'entropy'],
            'clf__max_depth': (5, 10, 25, 50, 100),
            'clf__min_samples_split': (1, 2, 3, 4),
            'clf__min_samples_leaf': (1, 2, 3, 4)
            }

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring='f1_weighted')
grid_search.fit(train_data, train_labels)

print('Best score: {0}%'.format(grid_search.best_score_*100))
print('Best parameters set:')

best_parameters = grid_search.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
    print('\t{0}: {1}'.format(param_name, best_parameters[param_name]))

predictions = grid_search.predict(test_data)
print(classification_report(test_labels, predictions))

Fitting 3 folds for each of 160 candidates, totalling 480 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.5/site-packages/ipykernel/__main__.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/L

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
            'n_neighbors': list(range(1, 8)),
            'weights': ['distance'],
            'algorithm': ['kd_tree'],
            }
knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid)
knn_mdl = knn_grid.fit(train_data, train_labels)

In [ ]:
print('Training score is', knn_mdl.score(train_data, train_labels))
print('Validation score is', knn_mdl.score(test_data, test_labels))
print('-'*80)
print('Classification report of training data:\n', classification_report(train_labels, knn_mdl.predict(train_data)))
print('-'*80)
print('Classification report of test data:\n', classification_report(test_labels, knn_mdl.predict(test_data)))